# FLASK 中的上下文语境

使用上下文语境(contexts)让某些变量可以暂时全局访问。

如果您来自 `Django` 背景，那么您可能已经注意到 `Flask` 中的 `view` 函数不接受 `request` 作为第一个参数。在 `Flask` 中，我们使用 `request` 对象访问传入请求中的数据，如下所示:

```python
from flask import Flask, request

@app.route('/')
def requestdata():
    return "Hello! Your IP is {} and you are using {}: ".format(request.remote_addr, request.user_agent)
```

上面的代码可能会给您一个印象，即 `request` 是一个全局对象，实际上它不是。如果 `request` 是一个全局对象，那么在一个多线程程序中，我们的应用程序将无法区分两个同时发出的请求，因为一个多线程程序共享线程中的所有变量。使用一种叫做 `Contexts` 的东西来使某些变量像全局变量一样工作，当你访问它们的时候，你就可以访问当前线程的对象。在术语集中，这样的变量被称为  `thread-locals` 线程局部变量。

## Flask 提供了两种 `Context` 上下文语境:

1. 应用程序上下文 Application Context.
2. 请求上下文 Request Context. 

应用程序上下文用于存储与应用程序通用的值，如数据库连接、配置等; 而请求上下文用于存储特定于每个请求的值。

应用程序上下文公开像 `current_app` 和 `g` 这样的对象。 `current_app` 指向处理请求的实例，而`g` 用于在请求处理期间临时存储数据。一旦设置了一个值，您就可以在任何视图函数中访问它。存储在 `g` 中的数据在每次请求后重置。

与应用程序上下文一样，请求上下文也公开 `request` 和 `session` 等对象。如前所述， `request` 对象包含有关当前 web 请求的信息， `session` 是一个类似字典的对象，用于存储在请求之间持久存储的值。

Flask 在收到请求时激活（或推送）应用程序和请求上下文，并在处理完请求时删除它们。
当应用程序上下文被推入时，它所公开的所有变量对线程来说都是可用的。
类似地，当请求上下文被推入时，它所公开的所有变量对线程来说都是可用的。
在视图函数中，您可以访问应用程序和请求上下文公开的所有对象，因此不必担心应用程序或请求上下文是否处于活动状态。
但是，如果您尝试在视图函数之外或在 pythonshell 中访问这些对象，则会出现错误。下面的 shell 会话演示了这一点:


In [2]:
# 如果您尝试在 视图函数 `a view function` 之外或在 `pythonshell` 中访问这些对象，则会出现错误。
from flask import Flask, request, current_app
request.method   # get the request method 

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.

`request.method` 返回请求中使用的 HTTP 方法，但是由于没有实际的 HTTP 请求，请求上下文是未激活的。

我们可以使用 Flask 实例的 `app_context()` 方法创建应用程序上下文。

In [3]:
from main2 import app
from flask import Flask, request, current_app

app_context = app.app_context()
app_context.push()

current_app.name

'main2'

前面的代码可以使用 `with` 语句简化如下:

In [4]:
from main2 import app
from flask import Flask, request, current_app

with app.app_context():   #With 语句是创建上下文的首选方法。
    print(current_app.name)

main2


With 语句是创建上下文的首选方法。

类似地，我们可以使用 Flask 实例的 `test_request_context()`方法创建请求上下文。需要记住的重要一点是，每当推送请求上下文时，应用程序上下文即被创建(如果尚未存在的话)。下面的 shell 会话演示了如何创建请求上下文:

In [5]:
from main2 import app
from flask import Flask, request, current_app

with app.test_request_context('/products'):  # The URL /products is arbitrarily chosen 任意选择的.
    print(request.path)
    print(request.method)
    print(current_app.name)

/products
GET
main2


URL链接 `/products` 是任意选择的

这就是你需要知道的关于 Flask 上下文的全部内容。